In [1]:
import time
import os
from sagemaker import get_execution_role, session
import boto3

region = boto3.Session().region_name
role = get_execution_role()
sm_client = boto3.client('sagemaker', region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


# Create Segmentation Model group

In [2]:
import time
model_package_group_name = "balloon-segemntation-nonprod-group"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Test Sample model package group"
}

In [3]:
model_package_group_name

'balloon-segemntation-nonprod-group'

In [4]:
create_model_package_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_package_group_response['ModelPackageGroupArn']))

ClientError: An error occurred (ValidationException) when calling the CreateModelPackageGroup operation: Model Package Group already exists: arn:aws:sagemaker:us-east-1:470086202700:model-package-group/balloon-segemntation-nonprod-group

In [5]:
## Change this to your model URI and mdoel .tar.gz use -cvzf for zipping model
model_url = "s3://sagemaker-us-east-1-470086202700/balloon_segemtaion_model/balloon_ins_segm.tar.gz"
#model_url = "s3://sagemaker-us-east-1-xxxxxxxx/balloon_segemtaion_model/balloon_ins_segm.tar.gz"
image_uri="470086202700.dkr.ecr.us-east-1.amazonaws.com/detectron2_segmentation_bt"
#image_uri="xxxxxxx.dkr.ecr.us-east-1.amazonaws.com/detectron2_segmentation"

In [6]:
modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": image_uri,
            "ModelDataUrl": model_url
         }
      ],
      "SupportedContentTypes": [ "json" ],
      "SupportedResponseMIMETypes": [ "json/csv" ],
   }
 }


In [7]:
create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Batch transform Model to detect balloons",
    "ModelApprovalStatus" : "PendingManualApproval"
}

In [8]:
create_model_package_input_dict.update(modelpackage_inference_specification)

In [9]:
create_model_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_model_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-1:470086202700:model-package/balloon-segemntation-nonprod-group/12


In [10]:
sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'balloon-segemntation-nonprod-group',
   'ModelPackageVersion': 12,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:470086202700:model-package/balloon-segemntation-nonprod-group/12',
   'ModelPackageDescription': 'Batch transform Model to detect balloons',
   'CreationTime': datetime.datetime(2024, 9, 10, 17, 58, 17, 580000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'},
  {'ModelPackageGroupName': 'balloon-segemntation-nonprod-group',
   'ModelPackageVersion': 11,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:470086202700:model-package/balloon-segemntation-nonprod-group/11',
   'ModelPackageDescription': 'Batch transform Model to detect balloons',
   'CreationTime': datetime.datetime(2024, 9, 10, 14, 38, 56, 640000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Approved'},
  {'ModelPackageGroupName': 'balloon-segemntation-no

## Approve Model 

In [11]:
model_package_update_input_dict = {
    "ModelPackageArn" : model_package_arn,
    "ModelApprovalStatus" : "Approved"
}
model_package_update_response = sm_client.update_model_package(**model_package_update_input_dict)

In [12]:
sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'balloon-segemntation-nonprod-group',
   'ModelPackageVersion': 12,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:470086202700:model-package/balloon-segemntation-nonprod-group/12',
   'ModelPackageDescription': 'Batch transform Model to detect balloons',
   'CreationTime': datetime.datetime(2024, 9, 10, 17, 58, 17, 580000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Approved'},
  {'ModelPackageGroupName': 'balloon-segemntation-nonprod-group',
   'ModelPackageVersion': 11,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:470086202700:model-package/balloon-segemntation-nonprod-group/11',
   'ModelPackageDescription': 'Batch transform Model to detect balloons',
   'CreationTime': datetime.datetime(2024, 9, 10, 14, 38, 56, 640000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Approved'},
  {'ModelPackageGroupName': 'balloon-segemntation-nonprod-group',

## Create Segmentation End Point

In [ ]:
import sagemaker
sagemaker_session=sagemaker.Session(default_bucket="sagemaker-us-east-1-470086202700")

In [ ]:
from sagemaker import ModelPackage
from time import gmtime, strftime
endpoint_name="balloon-segmentation-test"
model_package_arn="arn:aws:sagemaker:us-east-1:xxxxxx:model-package/balloon-segemntation-nonprod-group/2"

In [ ]:
model = ModelPackage(role=role, 
                     model_package_arn=model_package_arn, 
                     sagemaker_session=sagemaker_session)
endpoint_response=model.deploy(initial_instance_count=1, instance_type="ml.p3.2xlarge",endpoint_name=endpoint_name)

In [ ]:
model.endpoint_name

## Delete Model Package and Model Group

In [ ]:
sm_client.delete_model_package(ModelPackageName=model_package_arn)

In [ ]:
pkg_name={
   "ModelPackageGroupName": model_package_group_name
}
delete_model_package_group_response = sm_client.delete_model_package_group(**pkg_name)
delete_model_package_group_response

In [ ]:
sm_client.delete_endpoint(EndpointName=model.endpoint_name)